# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [100]:
# Passo 1: Pegar a cotação do dolar

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

# Entrar no google
navegador.get("https://www.google.com.br")

# Pesquisar o dolar

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação do dolár")

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Pegar o valor do dólar

cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

5.108899999999999


In [101]:
# Passo 2: Pegar a cotação do euro

navegador.get("https://www.google.com.br")

# Pesquisar o euro

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação do euro")

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Pegar o valor do euro

cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)


5.5454555050000005


In [102]:
# Passo 3: Pegar a cotação do ouro 

navegador.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")


print(cotacao_ouro)

navegador.quit()

312.10


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [103]:
# Passo 4: Importar a base de dados e ajustar 

import pandas as pd 

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [104]:
# Passo 5: Recalcular os preços 

# Atualizar preço da cotação

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# Atualizar o preço de compra

tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Atualizar o preço de venda

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.108900,5108.848911,1.40,7152.388475
1,Carro Renault,4500.00,Euro,5.545456,24954.549773,2.00,49909.099545
2,Notebook Dell,899.99,Dólar,5.108900,4597.958911,1.70,7816.530149
3,IPhone,799.00,Dólar,5.108900,4082.011100,1.70,6939.418870
4,Carro Fiat,3000.00,Euro,5.545456,16636.366515,1.90,31609.096379
5,Celular Xiaomi,480.48,Dólar,5.108900,2454.724272,2.00,4909.448544
6,Joia 20g,20.00,Ouro,312.100000,6242.000000,1.15,7178.300000


### Agora vamos exportar a nova base de preços atualizada

In [105]:
# Passo 6: Exportar a base atualizada

tabela.to_excel("Produtos Novo.xlsx", index=False)


In [106]:
!pip install selenium
